## Web Scrapping no site Reclame Aqui
Web Automation com Selenium

Este notebook é parte de um trabalho que tem como objetivo utilizar a base de reclamações do [Consumidor.gov](https://www.consumidor.gov.br/pages/principal/?1653940320598) para previsão da nota de avaliação do consumidor em relação à sua reclamação aberta no portal.

## Importação dos Módulos

In [73]:
import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

## Coleta dos Dados
#### Importação da Base do Consumidor.gov.br

In [2]:
# Importando os dados
consumidor = pd.read_csv('basecompleta2022-03.csv', sep = ';')

In [3]:
# Visualizando as primeiras 5 linhas
consumidor.head()

,Gestor,Canal de Origem,Região,UF,Cidade,Sexo,Faixa Etária,Ano Abertura,Mês Abertura,Data Abertura,...,Assunto,Grupo Problema,Problema,Como Comprou Contratou,Procurou Empresa,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,Análise da Recusa
0,Secretaria Nacional do Consumidor,Plataforma Web,S,RS,Morro Reuter,M,entre 41 a 50 anos,2022,1,13/01/2022,...,"Veículos Novos (motos, carros, caminhões)",Vício de Qualidade,Produto danificado / não funciona - Dificuldad...,Loja física,S,N,Finalizada não avaliada,Não Avaliada,NaN,Improcedente
1,Secretaria Nacional do Consumidor,Plataforma Web,S,RS,Porto Alegre,M,entre 41 a 50 anos,2022,1,13/01/2022,...,Aéreo,Cobrança / Contestação,Dificuldade / atraso na devolução de valores p...,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,Improcedente
2,Secretaria Nacional do Consumidor,Plataforma Web,S,RS,Porto Alegre,F,entre 31 a 40 anos,2022,1,13/01/2022,...,Aéreo,Cobrança / Contestação,Dificuldade / atraso na devolução de valores p...,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,Improcedente
3,Fundação de Proteção e Defesa do Consumidor,Plataforma Web,SE,SP,Santos,F,entre 31 a 40 anos,2022,1,14/01/2022,...,"Serviços na internet (provedor, hospedagem, ap...",Dados Pessoais e Privacidade,Vazamento de dados ou outros incidentes de seg...,Internet,S,S,Finalizada não avaliada,Não Avaliada,NaN,Improcedente
4,Fundação de Proteção e Defesa do Consumidor,Plataforma Web,SE,SP,Peruíbe,F,entre 21 a 30 anos,2022,1,14/01/2022,...,TV por Assinatura,Vício de Qualidade,Dificuldade para alterar/ ativar serviços,Não comprei / contratei,S,S,Finalizada não avaliada,Não Avaliada,NaN,Improcedente


In [4]:
# Obtendo o shape dos dados
consumidor.shape

# 143572 observações e 30 variáveis (colunas)

(143572, 30)

In [5]:
consumidor.columns

Index(['Gestor', 'Canal de Origem', 'Região', 'UF', 'Cidade', 'Sexo',
       'Faixa Etária', 'Ano Abertura', 'Mês Abertura', 'Data Abertura',
       'Data Resposta', 'Data Análise', 'Data Recusa', 'Data Finalização',
       'Prazo Resposta', 'Prazo Analise Gestor', 'Tempo Resposta',
       'Nome Fantasia', 'Segmento de Mercado', 'Área', 'Assunto',
       'Grupo Problema', 'Problema', 'Como Comprou Contratou',
       'Procurou Empresa', 'Respondida', 'Situação', 'Avaliação Reclamação',
       'Nota do Consumidor', 'Análise da Recusa'],
      dtype='object')

#### Web Scrapping Reclame Aqui
Para isso, utilizei uma lista com todos os valores únicos presentes na coluna Nome Fantasia. Porém, antes disso, foi necessário fazer uma limpeza nos dados para que fosse possível buscar seu correspondente no site do Reclame Aqui.

Aqueles que mesmo após a limpeza não foram encontrados, fiz a marcação com a tag 'Não Encontrado' para ser excluído depois.

In [6]:
# Tratando a coluna Nome Fantasia
nome_tratado = []

for item in consumidor['Nome Fantasia']:
    item = item.replace(".com", "-com")
    item = item.replace(' TV', '-TV')
    item = item.replace('Pontofrio-com', 'Ponto Frio Loja Online')
    
    
    nome_tratado.append(item)

In [7]:
# Verificando a quantidade de observações na lista
len(nome_tratado)

143572

In [8]:
# Inserindo os valores tratados no dataframe original
consumidor['Nome Tratado'] = nome_tratado

In [26]:
# Criando um novo df para armazenar apenas os valores únicos
df_scores = pd.DataFrame(consumidor['Nome Tratado'].unique(), columns = ['Nome Empresa'])
df_scores

,Nome Empresa
0,Chevrolet
1,Decolar-com
2,Facebook / Instagram
3,Claro-TV
4,Ponto Frio Loja Online
...,...
884,Systemcred
885,BMG Seguros
886,Banco SmartBank
887,Águas Colíder


In [28]:
# Criando uma nova coluna para alocar os scores e os nomes das empresas que aparecem no site do Reclame Aqui
df_scores['Score Reclame Aqui'] = ''
df_scores['Nome no Reclame Aqui'] = ''
df_scores

,Nome Empresa,Score Reclame Aqui,Nome no Reclame Aqui
0,Chevrolet,,
1,Decolar-com,,
2,Facebook / Instagram,,
3,Claro-TV,,
4,Ponto Frio Loja Online,,
...,...,...,...
884,Systemcred,,
885,BMG Seguros,,
886,Banco SmartBank,,
887,Águas Colíder,,


In [71]:
# Abrindo o navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\vieir\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [74]:
# Buscador do Score no Reclame Aqui

for i in range(len(df_scores['Nome Empresa'])):
        
    if "/" in df_scores['Nome Empresa'][i]:
            
        # Separando a string que possui mais de uma empresa
        item2 = df_scores['Nome Empresa'][i].split("/")
            
        try:
        
            for empresa in item2:

                scores = []

                # Acessando o site do reclame aqui
                driver.get('https://www.reclameaqui.com.br')

                # elemento HTML de busca do Google
                elem = driver.find_element(By.CLASS_NAME, 'input-auto-complete-search')
                
                # Colocando o nome da empresa no buscador
                elem.send_keys(str(empresa).lower().strip())
                time.sleep(2)
                
                # Localizando o elemento com o score
                elem2 = driver.find_element(By.CLASS_NAME, 'score')
                
                # Localizando o elemento com o nome da empresa
                elem3 = driver.find_element(By.CLASS_NAME, 'company-name')
                nome = BeautifulSoup(elem3.get_attribute("innerHTML"), "html.parser").get_text().strip()
                df_scores['Nome no Reclame Aqui'][i] = nome

                # Extraindo o texto
                score = BeautifulSoup(elem2.get_attribute("innerHTML"), "html.parser").get_text().split("/")[0]

                # Obtendo a média para as observações que possuem mais de uma empresa
                if score == '--':
                    score = 0
                    scores.append(score)
                else:
                    scores.append(float(score))

            df_scores['Score Reclame Aqui'][i] = np.mean(scores)
            
        except (NoSuchElementException, WebDriverException):
            
            df_scores['Score Reclame Aqui'][i] = 'Não Encontrado'
            
    else:
        
        try:
            
            # Acessando o site do reclame aqui
            driver.get('https://www.reclameaqui.com.br')

            # elemento HTML de busca do Google
            elem = driver.find_element(By.CLASS_NAME, 'input-auto-complete-search')

            # Colocando o nome da empresa no buscador
            elem.send_keys(str(df_scores['Nome Empresa'][i]).lower().strip())
            time.sleep(2)

            # Localizando o elemento com o score
            elem2 = driver.find_element(By.CLASS_NAME, 'score')
            
            # Localizando o elemento com o nome da empresa
            elem3 = driver.find_element(By.CLASS_NAME, 'company-name')
            nome = BeautifulSoup(elem3.get_attribute("innerHTML"), "html.parser").get_text().strip()
            df_scores['Nome no Reclame Aqui'][i] = nome

            # Extraindo o texto
            score = BeautifulSoup(elem2.get_attribute("innerHTML"), "html.parser").get_text().split("/")[0]
            
            # Obtendo nota da empresa
            if score == '--':
                score = 0
                
                df_scores['Score Reclame Aqui'][i] = float(score)
                
            else:
                
                df_scores['Score Reclame Aqui'][i] = float(score)
            
        except (NoSuchElementException, WebDriverException):
            
            df_scores['Score Reclame Aqui'][i] = 'Não Encontrado'

In [75]:
# Visualizando o dataframe
df_scores

,Nome Empresa,Score Reclame Aqui,Nome no Reclame Aqui
0,Chevrolet,7.2,Chevrolet
1,Decolar-com,4.9,Decolar
2,Facebook / Instagram,0.0,Instagram
3,Claro-TV,0.0,Claro TV
4,Ponto Frio Loja Online,6.7,Ponto (Ponto Frio) – Loja Online
...,...,...,...
884,Systemcred,0.0,SystemCred
885,BMG Seguros,0.0,Mg Seguros Vida E Previdência
886,Banco SmartBank,Não Encontrado,Americanas Marketplace
887,Águas Colíder,Não Encontrado,Americanas Marketplace


In [91]:
# Exportando a tabela
df_scores.to_excel('dados/scores21-04.xlsx')